In [1]:
%pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import json

import spotipy
from spotipy.oauth2 import SpotifyOAuth 
import os
import pandas as pd
from time import sleep

In [3]:
os.system("unzip dataset.zip")

256

The `sp.track`/`sp.tracks` endpoint gets information about the track (ex the artist, link to a preview, and a lot more).

The `sp.audio_features` endpoint gets the danceability/energy/acousticness/etc... ie the primary features we can use for our analysis.

The `sp.audio_analysis` endpoint contains more information about the song like timbre/mode/key/fadeout-duration/etc... more music theory-y features.

In [4]:
# Unsafe API Key Information OK because this is a burner acount... probably don't push to a public git repo though
API_USERNAME="0fdf4ffb7cad4650b9a22ea9cd6f4b8c"
API_KEY="e94b06ba10f844fb87f3c1b2018d79c3"
REDIRECT_URI="http://localhost:8080"

# Create API Access Client (May Require Sign In - Link should be logged in terminal...)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=API_USERNAME,
    client_secret=API_KEY,
    redirect_uri=REDIRECT_URI
))

In [5]:
data_stream = ""
with open("challenge_set.json", "r") as f:
    data_stream = f.read()
    
data_set = json.loads(data_stream)

In [6]:
track_uris = set()
for i,j in enumerate(data_set["playlists"]):
    if len(j["tracks"]) == 100:
        for track in j["tracks"]:
            track_uris.add((track["track_name"], track["artist_name"], track["track_uri"]))

In [7]:
chunk_size = 100
unique_tracks = list(track_uris)
chunks = np.arange(0, len(unique_tracks), chunk_size)
for chunk in chunks:
    print(f"Chunk {chunk} out of {chunks[-1]}")
    tracks = unique_tracks[chunk:chunk+chunk_size]
    name_author = []
    request_ids = []
    
    for track in tracks:
        name, author, req_id = track
        name_author.append((name, author))
        request_ids.append(req_id)
        
    try:
        resp = sp.audio_features(request_ids)
        df = pd.DataFrame(resp)
        df.to_csv(f"./fetched_chunks/{chunk}_{chunk+chunk_size}.csv")
    except:
        print(f"failed chunk {chunk}:{chunk+chunk_size}")
        with open("failures.txt", "a") as f:
            f.write(f"failed chunk {chunk}:{chunk+chunk_size}\n")
    finally:
        sleep(2)

Chunk 0 out of 53500
Chunk 100 out of 53500
Chunk 200 out of 53500
Chunk 300 out of 53500
Chunk 400 out of 53500
Chunk 500 out of 53500
Chunk 600 out of 53500
Chunk 700 out of 53500
Chunk 800 out of 53500
Chunk 900 out of 53500
Chunk 1000 out of 53500
Chunk 1100 out of 53500
Chunk 1200 out of 53500
Chunk 1300 out of 53500
Chunk 1400 out of 53500
Chunk 1500 out of 53500
Chunk 1600 out of 53500
Chunk 1700 out of 53500
Chunk 1800 out of 53500
Chunk 1900 out of 53500
Chunk 2000 out of 53500
Chunk 2100 out of 53500
Chunk 2200 out of 53500
Chunk 2300 out of 53500
Chunk 2400 out of 53500
Chunk 2500 out of 53500
Chunk 2600 out of 53500
Chunk 2700 out of 53500
Chunk 2800 out of 53500
Chunk 2900 out of 53500
Chunk 3000 out of 53500
Chunk 3100 out of 53500
Chunk 3200 out of 53500
Chunk 3300 out of 53500
Chunk 3400 out of 53500
Chunk 3500 out of 53500
Chunk 3600 out of 53500
Chunk 3700 out of 53500
Chunk 3800 out of 53500
Chunk 3900 out of 53500
Chunk 4000 out of 53500
Chunk 4100 out of 53500
Chun

### Patch and Record Missing Data

In [17]:
chunk = 8100
print(f"Chunk {chunk} out of {chunks[-1]}")
tracks = unique_tracks[chunk:chunk+chunk_size]
name_author = []
request_ids = []

for track in tracks:
    name, author, req_id = track
    name_author.append((name, author))
    request_ids.append(req_id)

resp = sp.audio_features(request_ids)
df = pd.DataFrame([x for x in resp if x is not None])
df.to_csv(f"./fetched_chunks/{chunk}_{chunk+chunk_size}.csv")

Chunk 8100 out of 53500


In [18]:
for i,j,k in zip(name_author,request_ids,  resp):
    if k is None:
        print(i,j)

('Best of 2015 (Album Mix)', 'Monstercat') spotify:track:4mjgNE8R31AzxWfPNGtVMf


In [19]:
chunk = 12100
print(f"Chunk {chunk} out of {chunks[-1]}")
tracks = unique_tracks[chunk:chunk+chunk_size]
name_author = []
request_ids = []

for track in tracks:
    name, author, req_id = track
    name_author.append((name, author))
    request_ids.append(req_id)

resp = sp.audio_features(request_ids)
df = pd.DataFrame([x for x in resp if x is not None])
df.to_csv(f"./fetched_chunks/{chunk}_{chunk+chunk_size}.csv")

Chunk 12100 out of 53500


In [22]:
for i,j,k in zip(name_author,request_ids,  resp):
    if k is None:
        print(i,j)

('Best of 2016 (Album Mix)', 'Monstercat') spotify:track:5CmMyiq4um2aFhiLZ1rPWo


### Try Loading all the downloaded data

In [39]:
with open("all_downloaded_data.csv", "w") as out:
    out.write(",danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature\n")
    for file in [x for x in os.listdir("./fetched_chunks") if x[-4:] == ".csv"]:
        with open("./fetched_chunks/"+file, "r") as f:
            for idx, line in enumerate(f.readlines()):
                if idx == 0:
                    continue
                out.write(line)

In [52]:
df = pd.read_csv("all_downloaded_data.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.set_index('uri')

In [53]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,
spotify:track:38uqMByifM0mkjcTbejlyR,0.375,0.294,8,-8.751,1,0.0329,0.928000,0.000000,0.1900,0.125,95.128,audio_features,38uqMByifM0mkjcTbejlyR,https://api.spotify.com/v1/tracks/38uqMByifM0m...,https://api.spotify.com/v1/audio-analysis/38uq...,161573,4
spotify:track:4kOTDj4SiZPkW1p96WMDNv,0.950,0.803,1,-4.195,1,0.1510,0.145000,0.000000,0.0748,0.860,117.000,audio_features,4kOTDj4SiZPkW1p96WMDNv,https://api.spotify.com/v1/tracks/4kOTDj4SiZPk...,https://api.spotify.com/v1/audio-analysis/4kOT...,213066,4
spotify:track:1c9LBfX0klXJscJAYDI0Ea,0.758,0.501,0,-7.051,1,0.0589,0.252000,0.000000,0.1910,0.397,78.042,audio_features,1c9LBfX0klXJscJAYDI0Ea,https://api.spotify.com/v1/tracks/1c9LBfX0klXJ...,https://api.spotify.com/v1/audio-analysis/1c9L...,194147,4
spotify:track:59iNA6gwthoTJ6IgpBGGHW,0.534,0.749,6,-5.562,0,0.0749,0.077600,0.000001,0.0906,0.671,179.975,audio_features,59iNA6gwthoTJ6IgpBGGHW,https://api.spotify.com/v1/tracks/59iNA6gwthoT...,https://api.spotify.com/v1/audio-analysis/59iN...,212160,4
spotify:track:4QCng4MDyUhEZ730Ma2Rjp,0.674,0.744,11,-5.196,1,0.2740,0.023800,0.000000,0.3300,0.624,154.086,audio_features,4QCng4MDyUhEZ730Ma2Rjp,https://api.spotify.com/v1/tracks/4QCng4MDyUhE...,https://api.spotify.com/v1/audio-analysis/4QCn...,262400,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:3SI9uxzQCSEfj7V1Qvf7nU,0.394,0.641,1,-7.645,1,0.3730,0.247000,0.000004,0.6560,0.193,165.120,audio_features,3SI9uxzQCSEfj7V1Qvf7nU,https://api.spotify.com/v1/tracks/3SI9uxzQCSEf...,https://api.spotify.com/v1/audio-analysis/3SI9...,389800,4
spotify:track:5DrRvw69AtJ0PwVpEp22NO,0.810,0.562,1,-4.593,1,0.1300,0.050700,0.000000,0.1220,0.179,119.892,audio_features,5DrRvw69AtJ0PwVpEp22NO,https://api.spotify.com/v1/tracks/5DrRvw69AtJ0...,https://api.spotify.com/v1/audio-analysis/5DrR...,241091,4
spotify:track:4zYBVaPx3EfZIUoNWFtdov,0.563,0.697,8,-6.127,0,0.0439,0.000575,0.000000,0.2240,0.653,148.040,audio_features,4zYBVaPx3EfZIUoNWFtdov,https://api.spotify.com/v1/tracks/4zYBVaPx3EfZ...,https://api.spotify.com/v1/audio-analysis/4zYB...,247408,4
